## Data Processes: Second Assignment (first part)

### Grupo:
Víctor Morcuende Castell, 47315589N

Guillermo Nájera Lavid, 70845359T

Javier Rocamora García, 20081979N

Antonio Ruiz García, 06601574E

In [30]:
import pandas as pd
from sklearn import preprocessing
from collections import defaultdict
from sklearn.impute import SimpleImputer
import math
import numpy as np

In [31]:
# Auxiliary functions

def diagnoseDate_to_ageDiagnosed(birthDate, diagnoseDate):
    age = pd.to_datetime(diagnoseDate).year-pd.to_datetime(birthDate).year
    return age

def deathDate_to_survivalTime(diagnosisDate, deathDate):
    survivalTime = pd.to_datetime(deathDate).year-pd.to_datetime(diagnosisDate).year
    if math.isnan(survivalTime) or survivalTime < 0:
        survivalTime = 100
    return survivalTime

def deathDate_to_survived(deathDate):
    survived = pd.to_datetime(deathDate).year
    if math.isnan(survived):
        survived = 1
    else: 
        survived = 0
    return survived

def deathDate_to_age(birthDate, deathDate):
    deathAge = pd.to_datetime(deathDate).year-pd.to_datetime(birthDate).year
    if math.isnan(deathAge):
        deathAge = 0
    return deathAge

def recurrence_year_to_recurrence_time(diagnosisDate, recurrence_year):
    recurrenceTime = recurrence_year-pd.to_datetime(diagnosisDate).year
    if math.isnan(recurrenceTime) or recurrenceTime < 0:
        recurrenceTime = 1000
    return recurrenceTime

In [32]:
df = pd.read_excel("breast_cancer_data.xlsx", index_col='ehr')

# Deleting unused column and duplicated data
df.pop('Unnamed: 0')
df = df.drop_duplicates(keep='first')

# Duplicating the DataFrame in order to obtain the numerical variables
df_num = pd.DataFrame(data=df, columns=df.columns, index=df.index)
df_num.pop('side')
df_num.pop('neoadjuvant')
df_num.pop('grade')
df_num.pop('invasive')
df_num.pop('er_positive')
df_num.pop('pr_positive')
df_num.pop('her2_positive')
df_num.pop('hist_type')

# Dividing the DataFrame into categorical and numerical variables
num_cols = df_num.columns.tolist()
df_cat = df.drop(num_cols, axis=1)

We delete the NULL values of the categorical variables by using the Simple Imputer

In [33]:
df_cat.side = df_cat.side.apply(lambda x: 'unknown' if (x != 'left' and x != 'right') else x)
df_cat.invasive = df_cat.invasive.apply(lambda x: 0 if x != 1 else x)

# Imputation of nulls in categorical columns using Simple Imputer
imp_cat = SimpleImputer(strategy='most_frequent')
columns = df_cat.columns
index = df_cat.index
df_cat = pd.DataFrame(imp_cat.fit_transform(df_cat), columns=columns, index=index)

# Transforming categorical values into numerical variables
df_cat.neoadjuvant = df_cat.neoadjuvant.apply(lambda x: 0.0 if x == 'no' else 1.0)

Now we are about to subsitute the categorical labels that are represented by strings with numerical values, in order to avoid working with Strings.

In [34]:
# Taking some variables out as they are already converted into numerical values
df_aux = pd.DataFrame(data=df_cat, columns=df_cat.columns, index=df_cat.index)
df_cat.pop('neoadjuvant')
df_cat.pop('invasive')
df_cat.pop('er_positive')
df_cat.pop('pr_positive')
df_cat.pop('her2_positive')
num_cols = df_cat.columns.tolist()
df_aux = df_aux.drop(num_cols, axis=1)

# Using OneHotEncoder
ohe = preprocessing.OneHotEncoder(sparse=False)
df_cat_ohe = pd.DataFrame(ohe.fit_transform(df_cat), 
                          columns=ohe.get_feature_names_out(df_cat.columns.tolist()),
                          index=df_cat.index)

# Merge both DataFrames (df_cat_ohe and df_aux)
df_cat_def = df_cat_ohe.join(other=df_aux)

We will swap birth_date, diagnosis_date and death_date with the age at which the patient was diagnosed, their age at the time of death if so, and how long they survived in this case. This will be of way more use to us when trying to use this data for any predictions

In [35]:
# Age at which the patient was diagnosed
ageDiagnosed = pd.Series(df_num.apply(lambda x: diagnoseDate_to_ageDiagnosed(x.birth_date, x.diagnosis_date), axis=1), name='age_diagnosed')

# Time of survival since diagnosis, 100 in case of full recovery
survivalTime = pd.Series(df_num.apply(lambda x: deathDate_to_survivalTime(x.diagnosis_date, x.death_date), axis=1), name='survival_time')

# We set "survived" column to be the target variable
class_col = pd.Series(df_num.apply(lambda x: deathDate_to_survived(x.death_date), axis=1), name='survived')

# Recurrence time for a patient
recurrenceTime = pd.Series(df_num.apply(lambda x: recurrence_year_to_recurrence_time(x.diagnosis_date, x.recurrence_year), axis=1), name='recurrence_time')

# Changing variables
df_num.pop('birth_date')
df_num.pop('diagnosis_date')
df_num.pop('death_date')
df_num.pop('recurrence_year')
df_num = pd.merge(left=df_num, right=ageDiagnosed, on='ehr')
df_num = pd.merge(left=df_num, right=survivalTime, on='ehr')
df_num = pd.merge(left=df_num, right=recurrenceTime, on='ehr')

We will treat differently the caesarean, abort and pregnancy labels, which should not be filled with data computed from the mean of the rest of the data

In [ ]:
df_num.apply(lambda x: 0 if math.isnan(x) else x)


We delete the NULL values of the numerical variables by using the Simple Imputer

In [36]:
# Imputation of nulls in categorical columns using Simple Imputer
imp_num = SimpleImputer(strategy='mean')
columns = df_num.columns
index = df_num.index
df_num_def = pd.DataFrame(imp_num.fit_transform(df_num), columns=columns, index=index)

df_num_def.menarche_age = df_num_def.menarche_age.apply(np.ceil)
df_num_def.menopause_age = df_num_def.menopause_age.apply(np.ceil)

print(df_num_def)

           ki67  menarche_age  menopause_age  pregnancy     abort  birth  \
ehr                                                                        
6849  19.582781          17.0           51.0    2.00000  0.000000    2.0   
268   19.582781          12.0           49.0    2.00000  0.000000    2.0   
1458   0.000000          11.0           49.0    2.00000  0.000000    2.0   
2013  17.000000          13.0           49.0    1.81746  0.300885   -1.0   
1350  44.000000          14.0           49.0    3.00000  0.300885    3.0   
...         ...           ...            ...        ...       ...    ...   
191   10.000000          13.0           53.0    2.00000  0.000000   -1.0   
6482   1.000000          11.0           53.0    1.81746  0.300885   -1.0   
2564  20.000000          11.0           49.0    0.00000  0.000000    0.0   
2730  20.000000          13.0           48.0    0.00000  0.000000    0.0   
2376  10.000000          14.0           49.0    2.00000  0.000000    2.0   

      caesa

Now that all variables are numerical and do not have missing values, we can merge the categorical and numerical variables

In [37]:
df_preprocessed = pd.merge(left=df_cat_def, right=df_num_def, on='ehr')
print(df_preprocessed)

df_preprocessed.to_excel("output.xlsx")

      side_left  side_right  side_unknown  grade_1.0  grade_2.0  grade_3.0  \
ehr                                                                          
6849        0.0         0.0           1.0        1.0        0.0        0.0   
268         0.0         0.0           1.0        0.0        1.0        0.0   
1458        0.0         0.0           1.0        1.0        0.0        0.0   
2013        0.0         0.0           1.0        0.0        0.0        1.0   
1350        0.0         0.0           1.0        0.0        1.0        0.0   
...         ...         ...           ...        ...        ...        ...   
191         0.0         0.0           1.0        0.0        1.0        0.0   
6482        0.0         0.0           1.0        0.0        1.0        0.0   
2564        0.0         0.0           1.0        0.0        0.0        1.0   
2730        0.0         0.0           1.0        0.0        1.0        0.0   
2376        0.0         0.0           1.0        1.0        0.0 

Now we are going to eliminate the labels that do not give relevant information but would increase the computational cost of our data treatments.
e.g: Columns with too many different values, such as descriptions or names.